In [13]:
from __future__ import annotations
from typing import Optional, TypedDict
from pydantic import BaseModel
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langgraph.graph import StateGraph

In [ ]:
class CandidateDocument(BaseModel):
    name: str
    summary: str

class EvaluationScores(BaseModel):
    team_trustworthiness: int
    customer_feedback: int
    risk_analysis: int
    founder_commitment: int

class MarketAnalysis(BaseModel):
    competitive_score: Optional[float] = None
    market_score: Optional[float] = None
    competitive_reasoning: Optional[str] = None
    market_reasoning: Optional[str] = None

class InvestmentDecision(BaseModel):
    judgement: Optional[str] = None      # "통과" | "불통과"
    reasoning: Optional[str] = None      # 총점·근거

class GraphState(BaseModel):
    user_query: str
    domain: Optional[str] = None
    candidates_documents: Optional[List[CandidateDocument]] = []
    evaluation_scores: Optional[EvaluationScores] = None
    evaluation_summary: Optional[str] = None
    market_analysis: Optional[MarketAnalysis] = None
    investment_decision: Optional[InvestmentDecision] = None
    final_report: Optional[str] = None


In [15]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

In [ ]:
investment_prompt = ChatPromptTemplate.from_template("""
당신은 AI 스타트업 투자 평가 전문가입니다.

아래는 평가 점수(10점 만점) 및 요약 설명입니다.
총점을 계산하고 다음 중 하나로 판단하세요:
① 통과 (총점 ≥ 80) ② 조건부 (60 ≤ 총점 < 80) ③ 불통과 (총점 < 60)

[점수]
- 창업자 신뢰도: {team_trustworthiness}
- 고객 피드백: {customer_feedback}
- 리스크 분석(높을수록 리스크 낮음): {risk_analysis}
- 창업자 헌신도: {founder_commitment}
- 시장성 점수: {market_score}
- 경쟁력 점수: {competitive_score}

[추가 설명]
{evaluation_summary}

[형식]
판단: <통과/조건부/불통과>
사유: <한두 문장 근거>
""")


In [ ]:
def investment_decision_node(state: GraphState) -> GraphState:
    """
    Bessemer Checklist 점수(0~10)를 합산하여
    총점 80 이상이면 '통과', 아니면 '불통과'로 판단.
    reasoning 에는 총점과 항목별 근거를 포함.
    """

    if state.evaluation_scores is None or state.market_analysis is None:
        raise ValueError("필수 평가 항목(evaluation_scores, market_analysis)이 부족합니다.")

    es = state.evaluation_scores
    ma = state.market_analysis

    # 1. 총점 계산
    total = (
        es.team_trustworthiness +
        es.customer_feedback +
        es.risk_analysis +
        es.founder_commitment +
        (ma.competitive_score or 0) +
        (ma.market_score or 0)
    )

    # 2. 판단
    judgement = "통과" if total >= 80 else "불통과"

    # 3. reasoning 생성
    reasoning_lines = [
        f"총점: {total}점 (80점 이상이면 '통과')",
        f"- 창업자 신뢰도: {es.team_trustworthiness}",
        f"- 고객 피드백: {es.customer_feedback}",
        f"- 리스크 분석(높을수록 리스크 낮음): {es.risk_analysis}",
        f"- 창업자 헌신도: {es.founder_commitment}",
        f"- 경쟁력 점수: {ma.competitive_score or 0}",
        f"- 시장성 점수: {ma.market_score or 0}",
    ]
    if ma.competitive_reasoning:
        reasoning_lines.append(f"  · 경쟁력 근거: {ma.competitive_reasoning}")
    if ma.market_reasoning:
        reasoning_lines.append(f"  · 시장성 근거: {ma.market_reasoning}")

    state.investment_decision = InvestmentDecision(
        judgement=judgement,
        reasoning="\n".join(reasoning_lines)
    )
    return state

In [ ]:
def investment_decision_node_llm(state: GraphState) -> GraphState:
    if not (state.get("evaluation_scores") and state.get("market_analysis")):
        raise ValueError("필수 평가 항목이 부족합니다.")

    # 프롬프트 입력값 구성
    scores = state["evaluation_scores"]
    market = state["market_analysis"]

    prompt_input = {
        "team_trustworthiness": scores.team_trustworthiness,
        "customer_feedback": scores.customer_feedback,
        "risk_analysis": scores.risk_analysis,
        "founder_commitment": scores.founder_commitment,
        "competitive_score": market.competitive_score or 0,
        "competitive_reasoning": market.competitive_reasoning or "없음",
        "market_score": market.market_score or 0,
        "market_reasoning": market.market_reasoning or "없음"
    }

    # LLM 실행
    chain = investment_prompt | llm | StrOutputParser()
    result = chain.invoke(prompt_input)

    # 결과 파싱
    judgement = None
    reasoning = None
    for line in result.splitlines():
        if line.startswith("판단:"):
            # ⛔ 잘못: line.replace(a"판단:", "")
            # ✅ 수정:
            judgement = line.replace("판단:", "").strip()
        elif line.startswith("사유:"):
            reasoning = line.replace("사유:", "").strip()

    # 상태에 반영
    state["investment_decision"] = InvestmentDecision(
        judgement=judgement,
        reasoning=reasoning
    )
    return state

In [27]:
builder = StateGraph(AgentState)

# 투자 판단 노드만 단독 테스트
builder.add_node("decide_investment", investment_decision_node_llm)
builder.set_entry_point("decide_investment")
builder.set_finish_point("decide_investment")

graph = builder.compile()

# 더미 입력
dummy_state: AgentState = {
    "evaluation_scores": EvaluationScores(
        team_trustworthiness=9,
        customer_feedback=8,
        risk_analysis=6,
        founder_commitment=9
    ),
    "market_analysis": MarketAnalysis(
        competitive_score=8.5,
        market_score=9.5,
        competitive_reasoning="특허 기반 AI 진단",
        market_reasoning="TAM 70B 이상"
    )
}

output = graph.invoke(dummy_state)
print(output["investment_decision"])

judgement='조건부' reasoning='초기 고객 반응과 리스크 수준이 낮은 점수를 받아 총점이 60점 미만으로 조건부 판단되었습니다. 그러나 창업자 신뢰도와 시장성이 높은 평가를 받아 투자 가능성이 있는 스타트업으로 평가됩니다.'
